In [1]:
import numpy as np
import xarray as xr
import xrlint.all as xrl

# XRLint

---

### Basic API Usage

In [2]:
xrl.version

'0.0.3'

In [3]:
nx = 2
ny = 3
nt = 4
ds = xr.Dataset(
    attrs=dict(title="SST-Climatology Subset"),
    coords={
        "x": xr.DataArray(
            np.linspace(-180, 180, nx),
            dims="x", 
            attrs={"units": "degrees"}
        ),
        "y": xr.DataArray(
            np.linspace(-90, 90, ny),
            dims="y", 
            attrs={"units": "degrees"}
        ),
        "time": xr.DataArray(
            [2010 + y for y in range(nt)], 
            dims="time", 
            attrs={"units": "years"}
        ),
        "spatial_ref": xr.DataArray(
            0,
            attrs={
                "grid_mapping_name": "latitude_longitude",
                "semi_major_axis": 6371000.0,
                "inverse_flattening": 0,
            },
        ),
    },
    data_vars={
        "sst": xr.DataArray(
            np.random.random((nt, ny, nx)), 
            dims=["time", "y", "x"], 
            attrs={"units": "kelvin", "grid_mapping": "spatial_ref"}
        ),
        "sst_anomaly": xr.DataArray(
            np.random.random((nt, ny, nx)), 
            dims=["time", "y", "x"], 
            attrs={"units": "kelvin", "grid_mapping": "spatial_ref"}
        )
    },
)

In [4]:
linter = xrl.new_linter(recommended=True)

In [5]:
linter.verify_dataset(ds)

Result(config=Config(name=None, files=None, ignores=None, linter_options=None, opener_options=None, processor=None, plugins={'__core__': Plugin(meta=PluginMeta(name='__core__', version='0.0.1'), configs={'recommended': Config(name='recommended', files=None, ignores=None, linter_options=None, opener_options=None, processor=None, plugins=None, rules={'coords-for-dims': RuleConfig(severity=2, args=(), kwargs={}), 'dataset-title-attr': RuleConfig(severity=1, args=(), kwargs={}), 'grid-mappings': RuleConfig(severity=2, args=(), kwargs={}), 'no-empty-attrs': RuleConfig(severity=1, args=(), kwargs={}), 'var-units-attr': RuleConfig(severity=1, args=(), kwargs={})}, settings=None), 'all': Config(name='all', files=None, ignores=None, linter_options=None, opener_options=None, processor=None, plugins=None, rules={'coords-for-dims': RuleConfig(severity=2, args=(), kwargs={}), 'dataset-title-attr': RuleConfig(severity=2, args=(), kwargs={}), 'grid-mappings': RuleConfig(severity=2, args=(), kwargs={}), 'no-empty-attrs': RuleConfig(severity=2, args=(), kwargs={}), 'var-units-attr': RuleConfig(severity=2, args=(), kwargs={})}, settings=None)}, rules={'coords-for-dims': Rule(meta=RuleMeta(name='coords-for-dims', version='1.0.0', description='Dimensions of data variables should have corresponding coordinates.', docs_url=None, schema=None, type='problem'), op_class=<class 'xrlint.plugins.core.rules.coords_for_dims.CoordsForDims'>), 'dataset-title-attr': Rule(meta=RuleMeta(name='dataset-title-attr', version='1.0.0', description='Datasets should be given a non-empty title.', docs_url=None, schema=None, type='suggestion'), op_class=<class 'xrlint.plugins.core.rules.dataset_title_attr.DatasetTitleAttr'>), 'grid-mappings': Rule(meta=RuleMeta(name='grid-mappings', version='1.0.0', description='Grid mappings, if any, shall have valid grid mapping coordinate variables.', docs_url=None, schema=None, type='problem'), op_class=<class 'xrlint.plugins.core.rules.grid_mappings.GridMappings'>), 'no-empty-attrs': Rule(meta=RuleMeta(name='no-empty-attrs', version='1.0.0', description='Every dataset element should have metadata that describes it.', docs_url=None, schema=None, type='suggestion'), op_class=<class 'xrlint.plugins.core.rules.no_empty_attrs.NoEmptyAttrs'>), 'var-units-attr': Rule(meta=RuleMeta(name='var-units-attr', version='1.0.0', description="Every variable should have a valid 'units' attribute.", docs_url=None, schema=None, type='suggestion'), op_class=<class 'xrlint.plugins.core.rules.var_units_attr.VarUnitsAttr'>)}, processors={})}, rules={'coords-for-dims': RuleConfig(severity=2, args=(), kwargs={}), 'dataset-title-attr': RuleConfig(severity=1, args=(), kwargs={}), 'grid-mappings': RuleConfig(severity=2, args=(), kwargs={}), 'no-empty-attrs': RuleConfig(severity=1, args=(), kwargs={}), 'var-units-attr': RuleConfig(severity=1, args=(), kwargs={})}, settings=None), file_path='<dataset>', messages=[], fixable_error_count=0, fixable_warning_count=0, error_count=0, fatal_error_count=0, warning_count=0)

In [6]:
invalid_ds = ds.copy()
invalid_ds.attrs = {}
invalid_ds.sst.attrs["units"] = 1
invalid_ds["sst_avg"] = xr.DataArray(
    np.random.random((nx, ny)), 
    dims=["x", "y"], 
    attrs={"units": "kelvin"}
)

In [7]:
linter.verify_dataset(invalid_ds)

dataset,warn,Missing 'title' attribute in dataset.,dataset-title-attr
dataset.attrs,warn,"Missing metadata, attributes are empty.",no-empty-attrs
dataset.data_vars['sst'],warn,Invalid 'units' attribute in variable 'sst'.,var-units-attr


Pass the configuration of rules via `rules`, which maps rule names to rule configurations.
A rule configuration is either a _severity_, or a list where the first element is a rule 
_severity_ and subsequent elements are rule arguments: 

- _severity_
- `[`_severity_`]`
- `[`_severity_`,` _arg-1 | kwargs_ `]`
- `[`_severity_`,` _arg-1_`,` _arg-2_`,` ...`,` _arg-n | kwargs_`]`

Here, _severity_ is either a

- one of `"error"`, `"warn"`, `"off"` or 
- one of `2` (error), `1` (warn), `0` (off)

In [8]:
linter = xrl.new_linter(
    rules={
        "no-empty-attrs": "warn",
        "dataset-title-attr": "warn",
        "grid-mappings": "error",
        "var-units-attr": "error",
        "xcube/cube-dims-order": "off",
        "xcube/single-grid-mapping": "error",
    }
)

In [9]:
linter.verify_dataset(invalid_ds)

dataset.attrs,warn,"Missing metadata, attributes are empty.",no-empty-attrs
dataset,warn,Missing 'title' attribute in dataset.,dataset-title-attr
dataset.data_vars['sst'],error,Invalid 'units' attribute in variable 'sst'.,var-units-attr
,error,unknown plugin 'xcube',xcube/cube-dims-order
,error,unknown plugin 'xcube',xcube/single-grid-mapping


---

### Configure Plugins

In [10]:
from xrlint.plugins.core import export_plugin 

core_plugin = export_plugin()

linter = xrl.Linter(
    plugins={
        "humpty-dumpty": core_plugin
    }, 
    rules={
        "humpty-dumpty/no-empty-attrs": "warn",
        "humpty-dumpty/dataset-title-attr": "error",
        "humpty-dumpty/var-units-attr": "warn"
    }
)

In [11]:
linter.verify_dataset(invalid_ds)

dataset.attrs,warn,"Missing metadata, attributes are empty.",humpty-dumpty/no-empty-attrs
dataset,error,Missing 'title' attribute in dataset.,humpty-dumpty/dataset-title-attr
dataset.data_vars['sst'],warn,Invalid 'units' attribute in variable 'sst'.,humpty-dumpty/var-units-attr


---

### XRLint objects

By default, a `Linter` has no configuration.

In [12]:
linter = xrl.Linter()

In [13]:
linter.config.plugins is None

True

In [14]:
linter.config.rules is None

True

The `new_linter()` function returns a `Linter` pre-configured with builtin plugins and their recommended rules.

In [15]:
linter = xrl.new_linter()

In [16]:
list(linter.config.plugins.keys())

['__core__']

In [17]:
linter.config.rules

If the `new_linter()` function is called with `recommended=False` it still has the builtin plugins, but without any rule configurations.

In [18]:
linter = xrl.new_linter(recommended=False)

In [19]:
list(linter.config.plugins.keys())

['__core__']

In [20]:
linter.config.rules is None

True

---

### XRLint CLI

In [21]:
!xrlint --help

Usage: xrlint [OPTIONS] [FILES]...

  Validate the given dataset FILES.

  Reads configuration from `xrlint.config.*` if file exists and unless `--no-
  default-config` is set or `--config PATH` is provided. Then validates each
  dataset in FILES against the configuration. The validation result is dumped
  to standard output if not otherwise stated by `--output-file PATH`. The
  output format is `simple`. Other inbuilt formats are `json` and `html` which
  can by setting the `--format NAME` option.

Options:
  --no-default-config     Disable use of default configuration from
                          xrlint_config.*
  -c, --config PATH       Use this configuration, overriding xrlint_config.*
                          config options if present
  --plugin MODULE         Specify plugins. MODULE is the name of Python module
                          that defines an 'export_plugin()' function.
  --rule SPEC             Specify rules. SPEC must have format '<rule-name>:
                     

In [22]:
import os
os.getcwd()

'C:\\Users\\norma\\Projects\\xrlint\\notebooks'

In [23]:
ds.to_zarr("valid.zarr", mode="w")

In [24]:
invalid_ds.to_zarr("invalid.zarr", mode="w")

In [25]:
!xrlint valid.zarr invalid.zarr


valid.zarr:
  error  No rules configured or applicable.  ]8;;https://bcdev.github.io/xrlint\https://bcdev.github.io/xrlint]8;;\

invalid.zarr:
  error  No rules configured or applicable.  ]8;;https://bcdev.github.io/xrlint\https://bcdev.github.io/xrlint]8;;\

2 errors



In [26]:
!xrlint valid.zarr invalid.zarr -f html

<div role="results">
<h3>Results</h3>
<div role="result">
<p role="file">valid.zarr:</p>
<table>
<tbody>
<tr><td></td><td>error</td><td>No rules configured or applicable.</td><td></td></tr>
</tbody>
</table><p role="summary">one error</p>
</div>
<hr/>
<div role="result">
<p role="file">invalid.zarr:</p>
<table>
<tbody>
<tr><td></td><td>error</td><td>No rules configured or applicable.</td><td></td></tr>
</tbody>
</table><p role="summary">one error</p>
</div>
</div>

